In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dataset = pd.read_csv("/kaggle/input/book-rating-prediction01/Train.csv",encoding="latin")
df = pd.DataFrame(dataset)
df.head()

In [ ]:
result = df.dtypes
for i in range(0, len(result)):
  print(df.columns[i], result[i])

In [ ]:
for i in df.columns:
    print(i,df[i].count())

In [ ]:
new_df = df
new_df.head()

In [ ]:
# new_df['year'] = new_df['date_added'].apply(lambda x: x[-4:] if isinstance(x, str) else None)
print(len(df))

In [ ]:
new_df['review_text'] = new_df['review_text'].str.lower()

In [ ]:
new_df['review_text']

### HTML Tag and urls removal

In [ ]:
import re

def remove_html_tags_and_urls(text):
    clean = re.sub(r'[\t\n]', '', text)
    clean = re.sub(r'<.*?>', '', clean)
    clean = re.sub(r'https?://\S+|www\.\S+', '', clean)
    clean = re.sub(r'\d+', '', clean)
    return clean

In [ ]:
new_df['review_text'] = new_df['review_text'].apply(remove_html_tags_and_urls)

In [ ]:
new_df['review_text']

### Remove Punctuation

In [ ]:
import string
string.punctuation

In [ ]:
punc = '''!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'''

In [ ]:
def remove_punc(text):
    return text.translate(str.maketrans('','', punc))

In [ ]:
new_df['review_text'] = new_df['review_text'].apply(remove_punc)

In [ ]:
new_df['review_text']

### Stopwords Removal

In [ ]:
from nltk.corpus import stopwords

In [ ]:
stopwords.words('english')

In [ ]:
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

In [ ]:
new_df['review_text'] = new_df['review_text'].apply(remove_stopwords)

In [ ]:
new_df['review_text']

### Tokenization and Stemming

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize

In [ ]:
ps = PorterStemmer()
def stemng(text):
    token = word_tokenize(text)
    return " ".join(ps.stem(word) for word in token)

In [ ]:
new_df['review_text'].dtypes

In [ ]:
new_df['review_text'] = new_df['review_text'].apply(stemng)

In [ ]:
new_df['review_text']

### Model

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import scipy.sparse as sp


# Split the data into features and target
X = new_df[['review_text', 'n_votes', 'n_comments']]
y = new_df['rating']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# TF-IDF vectorization for text data
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train['review_text'])
X_test_tfidf = vectorizer.transform(X_test['review_text'])

# Combine TF-IDF vectors with numerical columns
X_train_combined = sp.hstack((X_train_tfidf, X_train[['n_votes', 'n_comments']].values))
X_test_combined = sp.hstack((X_test_tfidf, X_test[['n_votes', 'n_comments']].values))

# Fit logistic regression model (multinomial)
logistic_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
logistic_model.fit(X_train_combined, y_train)

# Predict on the test set
predictions = logistic_model.predict(X_test_combined)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import scipy.sparse as sp


# Split the data into features and target
X = new_df[['review_text', 'n_votes', 'n_comments']]
y = new_df['rating']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# TF-IDF vectorization for text data
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train['review_text'])
X_test_tfidf = vectorizer.transform(X_test['review_text'])

# Combine TF-IDF vectors with numerical columns
X_train_combined = sp.hstack((X_train_tfidf, X_train[['n_votes', 'n_comments']].values))
X_test_combined = sp.hstack((X_test_tfidf, X_test[['n_votes', 'n_comments']].values))

# Fit Naive Bayes (Multinomial) model
nb_model = MultinomialNB()
nb_model.fit(X_train_combined, y_train)

# Predict on the test set
predictions = nb_model.predict(X_test_combined)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import scipy.sparse as sp


# Split the data into features and target
X = new_df[['review_text', 'n_votes', 'n_comments']]
y = new_df['rating']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# TF-IDF vectorization for text data
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train['review_text'])
X_test_tfidf = vectorizer.transform(X_test['review_text'])

# Combine TF-IDF vectors with numerical columns
X_train_combined = sp.hstack((X_train_tfidf, X_train[['n_votes', 'n_comments']].values))
X_test_combined = sp.hstack((X_test_tfidf, X_test[['n_votes', 'n_comments']].values))

# Fit SVM model
svm_model = SVC(kernel='linear')  # You can choose different kernels like 'rbf', 'poly', etc.
svm_model.fit(X_train_combined, y_train)

# Predict on the test set
predictions = svm_model.predict(X_test_combined)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import scipy.sparse as sp



# Split the data into features and target
X = new_df[['review_text', 'n_votes', 'n_comments']]
y = new_df['rating']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# TF-IDF vectorization for text data
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train['review_text'])
X_test_tfidf = vectorizer.transform(X_test['review_text'])

# Combine TF-IDF vectors with numerical columns
X_train_combined = sp.hstack((X_train_tfidf, X_train[['n_votes', 'n_comments']].values))
X_test_combined = sp.hstack((X_test_tfidf, X_test[['n_votes', 'n_comments']].values))

# Fit Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_combined, y_train)

# Predict on the test set
predictions = rf_model.predict(X_test_combined)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import scipy.sparse as sp


# Split the data into features and target
X = new_df[['review_text', 'n_votes', 'n_comments']]
y = new_df['rating']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# TF-IDF vectorization for text data
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train['review_text'])
X_test_tfidf = vectorizer.transform(X_test['review_text'])

# Combine TF-IDF vectors with numerical columns
X_train_combined = sp.hstack((X_train_tfidf, X_train[['n_votes', 'n_comments']].values))
X_test_combined = sp.hstack((X_test_tfidf, X_test[['n_votes', 'n_comments']].values))

# Fit KNN model
knn_model = KNeighborsClassifier(n_neighbors=750)  # Adjust the number of neighbors as needed
knn_model.fit(X_train_combined, y_train)

# Predict on the test set
predictions = knn_model.predict(X_test_combined)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import scipy.sparse as sp


# Split the data into features and target
X = new_df[['review_text', 'n_votes', 'n_comments']]
y = new_df['rating']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# TF-IDF vectorization for text data
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train['review_text'])
X_test_tfidf = vectorizer.transform(X_test['review_text'])

# Combine TF-IDF vectors with numerical columns
X_train_combined = sp.hstack((X_train_tfidf, X_train[['n_votes', 'n_comments']].values))
X_test_combined = sp.hstack((X_test_tfidf, X_test[['n_votes', 'n_comments']].values))

# Fit MLPClassifier (Neural Network)
mlp_model = MLPClassifier(hidden_layer_sizes=(100, ), max_iter=1000, random_state=42)
mlp_model.fit(X_train_combined, y_train)

# Predict on the test set
predictions = mlp_model.predict(X_test_combined)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)


In [ ]:
import pickle 

with open('model_ppkl', 'wb') as files:
    pickle.dump(new_df, files)